In [1]:
import pandas as pd
import numpy as np
import openmatrix as omx
import os

_join = os.path.join

In [2]:
# create a empty dataframe of all OD combinations and time period
import pandas as pd
from itertools import product

numbers_1_to_3332 = range(1, 3333)
combinations_1_to_5 = range(1, 6)
#combinations_strings = ['am', 'md', 'pm', 'ev', 'ea']

combinations = list(product(numbers_1_to_3332, numbers_1_to_3332))

df = pd.DataFrame(combinations, columns=['orig_taz', 'dest_taz'])
df.head()

,orig_taz,dest_taz
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5


In [3]:
geo_cwk = pd.read_csv(r"common_data\geographies.csv")

In [4]:
geo_cwk

,taz,rdm_zones,super_district,county
0,1,San Francisco_43,4,1
1,2,San Francisco_37,3,1
2,3,San Francisco_24,1,1
3,4,San Francisco_24,1,1
4,5,San Francisco_24,1,1
...,...,...,...,...
3327,3328,Marin_02,34,9
3328,3329,Marin_02,34,9
3329,3330,Marin_01,33,9
3330,3331,Marin_01,33,9


In [5]:
geo_cwk = pd.read_csv(r"common_data\geographies.csv")
geo_cwk_rdm = dict(zip(geo_cwk['taz'], geo_cwk['rdm_zones']))
geo_cwk_cty = dict(zip(geo_cwk['taz'], geo_cwk['county']))

df['orig_rdm'] = df['orig_taz'].map(geo_cwk_rdm)
df['dest_cty'] = df['dest_taz'].map(geo_cwk_cty)

In [6]:
df['flag'] = 0

In [7]:
orig_rdm = ['Contra Costa_13', 'Contra Costa_14', 'Contra Costa_15'] 
dest_cty = [4, 5, 9, 1, 2, 3, 6, 8]

In [9]:
df.loc[(df['orig_rdm'].isin(orig_rdm)) & (df['dest_cty'].isin(dest_cty)), 'flag'] = 1

In [10]:
#create matrices 
num_zones = 3332
OD_full_index = pd.MultiIndex.from_product([range(1,num_zones + 1), range(1,num_zones + 1)])

r_region = df.groupby(['orig_taz', 'dest_taz'])['flag'].sum().reindex(OD_full_index, fill_value=0).unstack().values

In [11]:
r_region.shape

(3332, 3332)

In [19]:
r_region.sum()

284925

In [12]:
bl_region = np.where(r_region>0,0,1)

In [23]:
r_region.max()

1

In [12]:
#skims from r41
r40_dir = r'C:\MTC_tmpy\TM2_2050R40_R2_Run4\calibration_3332\SinglePathTM2_2050R40_R2_Run4_omx_09222023'
r40_skims = os.listdir(r40_dir)
bl_skims = r'C:\MTC_tmpy\TM2_2050R40_R2_Run4\calibration_3332\SinglePathTM2_2050Baseline_R2_Run7_omx_09222023'
output_dir = r"C:\MTC_tmpy\TM2_2050R40_R2_Run4\calibration_3332\modified_single_path_skims"

cores = ['DTIME']

In [13]:
for skim in r40_skims:
    
    r40_skim = omx.open_file(_join(r40_dir, skim))
    bl_skim = omx.open_file(_join(bl_skims, skim))

    new_skim = omx.open_file(_join(output_dir, skim),'w')

    for core in r40_skim.list_matrices():
        bl_core = np.array(bl_skim[core])
        r40_core = np.array(r40_skim[core])
        if core in cores:
            new_core = r40_core * bl_region + bl_core * r_region
            print('changing', core, new_core.sum(), bl_core.sum(), r40_core.sum())
        else:
            new_core = r40_core
            print('not changing', core, new_core.sum(), bl_core.sum(), r40_core.sum())

        new_skim[core] = new_core
    
    r40_skim.close()
    bl_skim.close()
    new_skim.close()

not changing BOARDS 14720601.0 14354917.0 14720601.0
not changing DDIST 4700990734.0 4806930728.0 4700990734.0
changing DTIME 26782243177.0 27240136318.0 26638224564.0
not changing FARE 19755632.0 19026412.0 19755632.0
not changing IVT 49814339472.0 50770371749.0 49814339472.0
not changing IVTCOM 13420086532.0 11708608851.0 13420086532.0
not changing IVTEXP 2151036461.0 2174683726.0 2151036461.0
not changing IVTFRY 942673714.0 1062956403.0 942673714.0
not changing IVTHVY 25819914285.0 27840307169.0 25819914285.0
not changing IVTLOC 5492111193.0 5758109254.0 5492111193.0
not changing IVTLRT 1988399386.0 2225636556.0 1988399386.0
not changing IWAIT 4433015154.0 4700149579.0 4433015154.0
not changing PIVTCOM 12458917859.0 11083457369.0 12458917859.0
not changing PIVTEXP 3474801264.0 3477035924.0 3474801264.0
not changing PIVTFRY 964767721.0 1087966486.0 964767721.0
not changing PIVTHVY 22402369319.0 23161018047.0 22402369319.0
not changing PIVTLOC 17798716299.0 18774816479.0 17798716299.0

not changing WAIT 6975413626.0 7032002869.0 6975413626.0
not changing WAUX 2234189424.0 2350137368.0 2234189424.0
not changing WEGR 15030055330.0 14701283129.0 15030055330.0
not changing XWAIT 2734480080.0 2637026041.0 2734480080.0
not changing BOARDS 17633816.0 17397063.0 17633816.0
not changing DDIST 8859917380.0 9029412920.0 8859917380.0
changing DTIME 40883736706.0 41269745793.0 40616561038.0
not changing FARE 39499259.0 41071696.0 39499259.0
not changing IVT 57177340341.0 58313929679.0 57177340341.0
not changing IVTCOM 15064509527.0 12561833457.0 15064509527.0
not changing IVTEXP 7726163863.0 7718199505.0 7726163863.0
not changing IVTFRY 254730148.0 278894131.0 254730148.0
not changing IVTHVY 27129106377.0 29575505747.0 27129106377.0
not changing IVTLOC 5471509302.0 6633858431.0 5471509302.0
not changing IVTLRT 1525243963.0 1540938587.0 1525243963.0
not changing IWAIT 5299378923.0 5782675728.0 5299378923.0
not changing PIVTCOM 12405670407.0 10190858847.0 12405670407.0
not changing

not changing PIVTLRT 3081437482.0 3309465570.0 3081437482.0
not changing PerceivedTime_BestPath 168646569836.0 169208375483.0 168646569836.0
not changing WACC 0.0 0.0 0.0
not changing WAIT 13003017926.0 13711498077.0 13003017926.0
not changing WAUX 4268663068.0 4553106286.0 4268663068.0
not changing WEGR 18743232440.0 18690636844.0 18743232440.0
not changing XWAIT 7011061691.0 7028453287.0 7011061691.0
not changing BOARDS 13294648.0 11856891.0 13294648.0
not changing DDIST 4690821377.0 5037702456.0 4690821377.0
changing DTIME 22227199460.0 23243830346.0 22228374972.0
not changing FARE 18318565.0 17304868.0 18318565.0
not changing IVT 40214330082.0 38418790325.0 40214330082.0
not changing IVTCOM 10430574909.0 8570756775.0 10430574909.0
not changing IVTEXP 621596290.0 578427459.0 621596290.0
not changing IVTFRY 716886930.0 713452852.0 716886930.0
not changing IVTHVY 21964231872.0 22429588341.0 21964231872.0
not changing IVTLOC 5421640239.0 5042890213.0 5421640239.0
not changing IVTLRT 10

not changing PIVTEXP 1614272676.0 1569695440.0 1614272676.0
not changing PIVTFRY 1835063317.0 2143347650.0 1835063317.0
not changing PIVTHVY 26136148175.0 25611747794.0 26136148175.0
not changing PIVTLOC 6372535921.0 6310968733.0 6372535921.0
not changing PIVTLRT 1922588147.0 2030673358.0 1922588147.0
not changing PerceivedTime_BestPath 104280178684.0 102687779748.0 104280178684.0
not changing WACC 11187806762.0 10697349435.0 11187806762.0
not changing WAIT 8737558033.0 8718265778.0 8737558033.0
not changing WAUX 4190799107.0 4244089135.0 4190799107.0
not changing WEGR 102939257.0 102860705.0 102939257.0
not changing XWAIT 2934230535.0 3144837700.0 2934230535.0
not changing BOARDS 19339064.0 18617015.0 19339064.0
not changing DDIST 5630214867.0 5917330251.0 5630214867.0
changing DTIME 28144825846.0 29083788857.0 28089088501.0
not changing FARE 24789269.0 24424331.0 24789269.0
not changing IVT 59268669647.0 58967325440.0 59268669647.0
not changing IVTCOM 13564732872.0 10562161603.0 1356

not changing IVTHVY 27862467701.0 30923426876.0 27862467701.0
not changing IVTLOC 9011008606.0 9315054798.0 9011008606.0
not changing IVTLRT 2285907469.0 2362329029.0 2285907469.0
not changing IWAIT 8792408960.0 8678848184.0 8792408960.0
not changing PIVTCOM 20072995943.0 16163834924.0 20072995943.0
not changing PIVTEXP 7720735243.0 7586427351.0 7720735243.0
not changing PIVTFRY 3499448235.0 4002661904.0 3499448235.0
not changing PIVTHVY 39372067445.0 41364685287.0 39372067445.0
not changing PIVTLOC 9457795299.0 9788586572.0 9457795299.0
not changing PIVTLRT 2540860569.0 2744013704.0 2540860569.0
not changing PerceivedTime_BestPath 142124687502.0 143814445552.0 142124687502.0
not changing WACC 14048624619.0 13803332219.0 14048624619.0
not changing WAIT 13894649636.0 14207377888.0 13894649636.0
not changing WAUX 6407373629.0 6820032238.0 6407373629.0
not changing WEGR 213015526.0 224802503.0 213015526.0
not changing XWAIT 5102240676.0 5528529704.0 5102240676.0
not changing BOARDS 212813

In [24]:
#skims from r41
r40_dir = r'C:\MTC_tmpy\TM2_2050R40_R2_Run4\calibration_3332\SinglePathTM2_2050R40_R2_Run4_omx_09222023'
r40_skims = os.listdir(r40_dir)
bl_skims = r'C:\MTC_tmpy\TM2_2050R40_R2_Run4\calibration_3332\SinglePathTM2_2050Baseline_R2_Run7_omx_09222023'
output_dir = r"C:\MTC_tmpy\TM2_2050R40_R2_Run4\calibration_3332\modified_single_path_skims2"

cores = ['DTIME']

In [25]:
cores = 'PerceivedTime_BestPath'

In [26]:
for skim in r40_skims:
    
    r40_skim = omx.open_file(_join(r40_dir, skim))
    bl_skim = omx.open_file(_join(bl_skims, skim))

    new_skim = omx.open_file(_join(output_dir, skim),'w')

    
    bl_dtime = np.array(bl_skim['DTIME'])
    r40_dtime = np.array(r40_skim['DTIME'])
    
    new_dtime = r40_dtime * bl_region + bl_dtime * r_region
    
    for core in bl_skim.list_matrices(): 
        bl_core = np.array(bl_skim[core])
        r40_core = np.array(r40_skim[core])
        
        if core in cores:
            new_core = (r40_core - r40_dtime) + new_dtime
            print('changing', core, new_core.sum(), bl_core.sum(), r40_core.sum())
        else:
            new_core = r40_core
            print('not changing', core, new_core.sum(), bl_core.sum(), r40_core.sum())

        new_skim[core] = new_core
    
    r40_skim.close()
    bl_skim.close()
    new_skim.close()

not changing BOARDS 14720601.0 14354917.0 14720601.0
not changing DDIST 4700990734.0 4806930728.0 4700990734.0
not changing DTIME 26638224564.0 27240136318.0 26638224564.0
not changing FARE 19755632.0 19026412.0 19755632.0
not changing IVT 49814339472.0 50770371749.0 49814339472.0
not changing IVTCOM 13420086532.0 11708608851.0 13420086532.0
not changing IVTEXP 2151036461.0 2174683726.0 2151036461.0
not changing IVTFRY 942673714.0 1062956403.0 942673714.0
not changing IVTHVY 25819914285.0 27840307169.0 25819914285.0
not changing IVTLOC 5492111193.0 5758109254.0 5492111193.0
not changing IVTLRT 1988399386.0 2225636556.0 1988399386.0
not changing IWAIT 4433015154.0 4700149579.0 4433015154.0
not changing PIVTCOM 12458917859.0 11083457369.0 12458917859.0
not changing PIVTEXP 3474801264.0 3477035924.0 3474801264.0
not changing PIVTFRY 964767721.0 1087966486.0 964767721.0
not changing PIVTHVY 22402369319.0 23161018047.0 22402369319.0
not changing PIVTLOC 17798716299.0 18774816479.0 177987162

not changing WAIT 6975413626.0 7032002869.0 6975413626.0
not changing WAUX 2234189424.0 2350137368.0 2234189424.0
not changing WEGR 15030055330.0 14701283129.0 15030055330.0
not changing XWAIT 2734480080.0 2637026041.0 2734480080.0
not changing BOARDS 17633816.0 17397063.0 17633816.0
not changing DDIST 8859917380.0 9029412920.0 8859917380.0
not changing DTIME 40616561038.0 41269745793.0 40616561038.0
not changing FARE 39499259.0 41071696.0 39499259.0
not changing IVT 57177340341.0 58313929679.0 57177340341.0
not changing IVTCOM 15064509527.0 12561833457.0 15064509527.0
not changing IVTEXP 7726163863.0 7718199505.0 7726163863.0
not changing IVTFRY 254730148.0 278894131.0 254730148.0
not changing IVTHVY 27129106377.0 29575505747.0 27129106377.0
not changing IVTLOC 5471509302.0 6633858431.0 5471509302.0
not changing IVTLRT 1525243963.0 1540938587.0 1525243963.0
not changing IWAIT 5299378923.0 5782675728.0 5299378923.0
not changing PIVTCOM 12405670407.0 10190858847.0 12405670407.0
not chan

not changing PIVTLRT 3081437482.0 3309465570.0 3081437482.0
changing PerceivedTime_BestPath 168773595705.0 169208375483.0 168646569836.0
not changing WACC 0.0 0.0 0.0
not changing WAIT 13003017926.0 13711498077.0 13003017926.0
not changing WAUX 4268663068.0 4553106286.0 4268663068.0
not changing WEGR 18743232440.0 18690636844.0 18743232440.0
not changing XWAIT 7011061691.0 7028453287.0 7011061691.0
not changing BOARDS 13294648.0 11856891.0 13294648.0
not changing DDIST 4690821377.0 5037702456.0 4690821377.0
not changing DTIME 22228374972.0 23243830346.0 22228374972.0
not changing FARE 18318565.0 17304868.0 18318565.0
not changing IVT 40214330082.0 38418790325.0 40214330082.0
not changing IVTCOM 10430574909.0 8570756775.0 10430574909.0
not changing IVTEXP 621596290.0 578427459.0 621596290.0
not changing IVTFRY 716886930.0 713452852.0 716886930.0
not changing IVTHVY 21964231872.0 22429588341.0 21964231872.0
not changing IVTLOC 5421640239.0 5042890213.0 5421640239.0
not changing IVTLRT 10

not changing PIVTEXP 1614272676.0 1569695440.0 1614272676.0
not changing PIVTFRY 1835063317.0 2143347650.0 1835063317.0
not changing PIVTHVY 26136148175.0 25611747794.0 26136148175.0
not changing PIVTLOC 6372535921.0 6310968733.0 6372535921.0
not changing PIVTLRT 1922588147.0 2030673358.0 1922588147.0
changing PerceivedTime_BestPath 104285961231.0 102687779748.0 104280178684.0
not changing WACC 11187806762.0 10697349435.0 11187806762.0
not changing WAIT 8737558033.0 8718265778.0 8737558033.0
not changing WAUX 4190799107.0 4244089135.0 4190799107.0
not changing WEGR 102939257.0 102860705.0 102939257.0
not changing XWAIT 2934230535.0 3144837700.0 2934230535.0
not changing BOARDS 19339064.0 18617015.0 19339064.0
not changing DDIST 5630214867.0 5917330251.0 5630214867.0
not changing DTIME 28089088501.0 29083788857.0 28089088501.0
not changing FARE 24789269.0 24424331.0 24789269.0
not changing IVT 59268669647.0 58967325440.0 59268669647.0
not changing IVTCOM 13564732872.0 10562161603.0 1356

not changing IVTHVY 27862467701.0 30923426876.0 27862467701.0
not changing IVTLOC 9011008606.0 9315054798.0 9011008606.0
not changing IVTLRT 2285907469.0 2362329029.0 2285907469.0
not changing IWAIT 8792408960.0 8678848184.0 8792408960.0
not changing PIVTCOM 20072995943.0 16163834924.0 20072995943.0
not changing PIVTEXP 7720735243.0 7586427351.0 7720735243.0
not changing PIVTFRY 3499448235.0 4002661904.0 3499448235.0
not changing PIVTHVY 39372067445.0 41364685287.0 39372067445.0
not changing PIVTLOC 9457795299.0 9788586572.0 9457795299.0
not changing PIVTLRT 2540860569.0 2744013704.0 2540860569.0
changing PerceivedTime_BestPath 142173633216.0 143814445552.0 142124687502.0
not changing WACC 14048624619.0 13803332219.0 14048624619.0
not changing WAIT 13894649636.0 14207377888.0 13894649636.0
not changing WAUX 6407373629.0 6820032238.0 6407373629.0
not changing WEGR 213015526.0 224802503.0 213015526.0
not changing XWAIT 5102240676.0 5528529704.0 5102240676.0
not changing BOARDS 21281375.0